### **Step 1: Reorientation of Scans to the MNI152 Atlas Reference**  

In this step, the **NIfTI (.nii.gz) scans** are reoriented to align with the **MNI152 atlas reference space**. Standardising the orientation is crucial, as tools like **HD-BET** require scans to follow a consistent spatial layout to ensure accurate skull-stripping.  

The script automatically processes all **NIfTI files** within the specified input folder and its subdirectories. Each reoriented scan is saved in the same location as the original, with the prefix **"reoriented_"** added to the filename. This ensures that the original scans remain unchanged while the modified versions are easily identifiable.  

Additionally, a **log file** is generated in the input folder, listing the processed scans along with their reorientation status. If a scan fails to reorient, it will be flagged in the log, and manual reorientation will be required using **3D Slicer** or an alternative imaging toolkit.

#### **Running the Reorientation Script**  
To perform this step, run the following command in the terminal:  
```bash
python reorientation.py "./data/example_input_images"
```

#### **Requirements: Installing FSL**  
Before running the script, ensure that **FSL (FMRIB Software Library)** is installed on your system. You can download it from the official website:  
🔗 [FSL Installation Guide](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/)  

Once downloaded, install it by running:  
```bash
python ~/Downloads/fslinstaller.py
```  
During installation, you’ll be asked to choose a location for FSL—either the default path or a custom directory.  

#### **Setting Up Environment Variables**  
After installation, you need to configure your environment by adding the following lines to your **~/.bashrc** or **~/.zshrc** file:  
```bash
export FSLDIR=/home/lorenagarcia-foncillasmacias/fsl  # Replace with your chosen install location
. ${FSLDIR}/etc/fslconf/fsl.sh
export PATH=${FSLDIR}/bin:$PATH
```
To apply the changes, run:  
```bash
source ~/.bashrc   # or source ~/.zshrc
```

This ensures FSL is correctly set up and accessible for processing the scans.

### **Step 2: Skull-Stripping with HD-BET**  

In this step, the **HD-BET** tool is used to perform skull-stripping, generating a brain mask and a dilated version of the mask to ensure no critical brain structures are excluded. To run the process, use a command similar to the one below in the terminal:  

```bash
python skull_stripping.py "./data/example_input_images" "./data/example_input_hdbet_processed"
```  

Here:  
- The **first argument** (`"./data/example_input_images"`) specifies the folder containing the reoriented scans.  
- The **second argument** (`"./data/example_input_hdbet_processed"`) defines the output directory where the processed scans will be saved.  

The output includes three types of files:  
- **`hd_bet`**: The skull-stripped brain scan produced by HD-BET.  
- **`hd_bet_mask`**: The binary brain mask, which is then dilated using a **7×7×7** structuring element.  
- **`hd_bet_dilated`**: The final skull-stripped scan after applying the dilated mask.  

A **log file** is also generated in the output folder, documenting each processed scan along with its status. It indicates whether the scan was successfully processed with **HD-BET** and whether the **dilated binary brain mask** and **dilated skull-stripped scan** were successfully created. 

### Why Apply Dilation?  
Dilation is performed to **ensure that all brain structures, including peripheral regions and tumours near the skull base, are retained**. This prevents the unintended removal of important anatomical features during subsequent processing.

### Requirements  
Before running this step, make sure you have the **HD-BET** library installed:  

```bash
pip install hd-bet
```  

For more details, refer to the [HD-BET GitHub repository](https://github.com/MIC-DKFZ/HD-BET) or check out the related [research paper](https://doi.org/10.1002/hbm.24750).

### **Step 3.a: Affine Registration with Reoriented Whole Scan and Defacing**

In this step, each **reoriented scan** is registered to a standard MNI152 template using **BRAINSFit**, ensuring alignment with a common reference space. If an existing transformation matrix (e.g., one generated in **3D Slicer**) is found, it will be used instead of performing automatic registration. During defacing, the obtained dilated brain mask and face mask are employed to allow for accurate defacing while preserving essential brain structures.

The registration script takes the following inputs:  
- **Target image**: The T1-w MNI152 template.  
- **Target face mask**: A pre-defined face mask in MNI152 space.  
- **Floating images**: The reoriented scans to be registered.  
- **Brain masks**: The dilated skull-stripped masks from **Step 2**.  
- **Existing transforms**: If available, previously computed transformation matrices.  

For each scan, the following steps are performed:  
1. **Check for existing outputs** – If a defaced version already exists, the script skips reprocessing.  
2. **Perform registration** – If no existing transformation is available, BRAINSFit registers the scan to the template.  
3. **Apply the transformation** – The computed transformation is used to warp the **target face mask** into the scan’s space.  
4. **Resample the face mask** – The transformed face mask is adjusted to match the scan's resolution.  
5. **Combine masks** – The warped **face mask** and the **brain mask** from Step 2 are merged to create a final defacing mask.  
6. **Apply the mask** – The defacing mask is applied to the scan, removing facial features while preserving brain structures.  

A **log file** is created to track scans that could not be processed. If any scans fail, they will need to be manually registered using **alternative tools like 3D Slicer** and defaced.

In [ ]:
# If required when running slicer kernel:
# import slicer
# slicer.util.pip_install("xxx") # change xxx to missing library

In [57]:
import os
import subprocess
import pandas as pd
from glob import glob
from natsort import natsorted
import numpy as np
import nibabel as nib
import json
from tqdm import tqdm
import csv

In [58]:
# define the target image for the registration and the corresponding face mask
target = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/mni_icbm152_t1_tal_nlin_sym_55_ext.nii")
target_face_mask = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/t1_mask.nii.gz")

# define the list of images that need to be defaced (only include the reoriented files)
floating_imgs = natsorted([f for f in glob("./data/example_input_images/*/*.nii.gz") if "reoriented_" in f])

# define a list of existing transforms (for example created with 3D Slicer) that are used instead of the automatic registration
# value should be None if no transform exists and automatic registration is to be used
existing_transform_paths = [os.path.join(os.path.dirname(f), "Transform_to_template.txt") for f in floating_imgs]
existing_transform_paths = [f if os.path.isfile(f) else "None" for f in existing_transform_paths ]

# list of output paths for each defaced image
destination_root = "./data/example_output_hdbet/output_brain_mask"
results_folder_paths = [os.path.join(destination_root, *f.split(os.sep)[-2:-1]) for f in floating_imgs]

# check that all lists have the same length
assert len(floating_imgs) == len(results_folder_paths) == len(existing_transform_paths), f"The lists have different lengths, {len(floating_imgs)=}, {len(results_folder_paths)=}, {len(existing_transform_paths)=}"

print(f"Found {len(floating_imgs)} images that need to be defaced")

# print example paths
print("Input images: ", floating_imgs[0:3])
print("Existing transforms: ", existing_transform_paths[0:3])
print("Output paths: ", results_folder_paths[0:3])

# path to BRAINSFit executable
BRAINSFit_bin_path = os.path.join("./BRAINSTools/BRAINSFit")

# path to BRAINSFit executable
BRAINSresample_bin_path = "./BRAINSTools/BRAINSResample"

Found 2 images that need to be defaced
Input images:  ['./data/example_input_images/IXI002-Guys-0828-T1/reoriented_IXI002-Guys-0828-T1.nii.gz', './data/example_input_images/IXI002-Guys-0828-T2/reoriented_IXI002-Guys-0828-T2.nii.gz']
Existing transforms:  ['None', 'None']
Output paths:  ['./data/example_output_hdbet/output_brain_mask/IXI002-Guys-0828-T1', './data/example_output_hdbet/output_brain_mask/IXI002-Guys-0828-T2']


In [59]:
# Create a dictionary to pair dilated skull-stripped scans (input for registration) and reorientated whole scans (input for defacing)
# List the the dilated brain masks
brain_masks = natsorted(glob("./data/example_input_hdbet_processed/*/hd_bet_mask_*.nii.gz"))

# Create a dictionary to store pairs of floating images and corresponding whole scans
floating_to_brain_mask = {}

# Iterate through the floating images and find the matching brain mask for each
for floating_img in floating_imgs:
    # Remove the 'reoriented_' prefix from the floating image filename
    base_name = os.path.basename(floating_img).replace("reoriented_", "")
    
    # Search for the corresponding binary mask
    for mask in brain_masks:
        if base_name in os.path.basename(mask):
            floating_to_brain_mask[floating_img] = mask
            break 
print(floating_to_brain_mask)

{'./data/example_input_images/IXI002-Guys-0828-T1/reoriented_IXI002-Guys-0828-T1.nii.gz': './data/example_input_hdbet_processed/IXI002-Guys-0828-T1/hd_bet_mask_IXI002-Guys-0828-T1.nii.gz', './data/example_input_images/IXI002-Guys-0828-T2/reoriented_IXI002-Guys-0828-T2.nii.gz': './data/example_input_hdbet_processed/IXI002-Guys-0828-T2/hd_bet_mask_IXI002-Guys-0828-T2.nii.gz'}


In [60]:
# Variable to store paths of files not defaced
not_processed_scans = []

# Iterate over each set of images, result folder paths, and existing transformation files using tqdm for progress tracking
for i, (floating, results_folder_path, ex_tfm) in enumerate(tqdm(list(zip(floating_imgs, results_folder_paths, existing_transform_paths)))):

        print(i)
        try:
            # Check if the masked version of the floating image already exists
            if os.path.isfile(os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))):
                # If the masked file already exists, print a message and skip processing this pair
                print("file exists. skipping ..... ")
                continue  # Skip to the next iteration (image pair)
        
            else:
                print(results_folder_path)
                # If the masked file does not exist, create the output directory for this image pair
                os.makedirs(results_folder_path, exist_ok=True)
        
                # Define paths for the affine transformation matrix and resampled floating image
                out_affine = os.path.join(results_folder_path, os.path.basename(floating.replace("nii.gz", "txt")))
                out_resampled_img_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_resampled.nii.gz"))
                
                # Load the target and floating images using nibabel (NIfTI format)
                target_nii = nib.load(target)
                floating_nii = nib.load(floating)
                
                # If there is an existing affine transformation, use it instead of running registration
                if os.path.isfile(ex_tfm):
                    print(f"Using existing initialTransform {ex_tfm} instead of running registration...")
                    out_affine = ex_tfm  # Use the existing affine transformation matrix
                else:
        
                    # If no existing affine transformation, perform image registration using BRAINSFit
                    print(f"Run registration with BRAINSFit...")
                    exit_status = os.system(f' "{BRAINSFit_bin_path}" ' +  # BRAINSFit executable path
                                    f'--fixedVolume "{target}" ' +  # Path to the fixed (target) volume
                                    f'--movingVolume "{floating}" ' +  # Path to the moving (floating) volume
                                    f'--outputVolume "{out_resampled_img_path}" ' +  # Path to the output resampled volume
                                    f'--outputTransform "{out_affine}" ' +  # Path to the output transformation matrix
                                    f'--samplingPercentage 0.1  ' +  # 10% of the data used for registration (lower values = faster, less accurate)
                                    f'--splineGridSize 14,10,12  ' +  # Grid size for spline transformation
                                    f'--initializeTransformMode useMomentsAlign  ' +  # Method to initialize the transformation
                                    f'--useRigid  ' +  # Apply rigid transformation (translation, rotation)
                                    f'--useAffine  ' +  # Apply affine transformation (scaling, shearing)
                                    f'--maskProcessingMode NOMASK  ' +  # Don't apply any masks during registration
                                    f'--medianFilterSize 0,0,0  ' +  # Disable median filtering (default 0)
                                    f'--removeIntensityOutliers 0  ' +  # Disable outlier removal
                                    f'--outputVolumePixelType float  ' +  # Set output pixel type to float
                                    f'--backgroundFillValue 0  ' +  # Background fill value set to 0 (empty space in images)
                                    f'--interpolationMode Linear  ' +  # Use linear interpolation for image resampling
                                    f'--numberOfIterations 1500  ' +  # Number of iterations for optimization
                                    f'--maximumStepLength 0.05  ' +  # Maximum step size for optimization
                                    f'--minimumStepLength 0.001  ' +  # Minimum step size for optimization
                                    f'--relaxationFactor 0.5  ' +  # Relaxation factor for optimization
                                    f'--translationScale 1000  ' +  # Scale for translation optimization
                                    f'--reproportionScale 1  ' +  # Scale for reproportioning optimization
                                    f'--skewScale 1  ' +  # Scale for skewness optimization
                                    f'--maxBSplineDisplacement 0  ' +  # Maximum displacement for B-spline transformation
                                    f'--fixedVolumeTimeIndex 0  ' +  # Time index for the fixed volume (0 means no time series)
                                    f'--movingVolumeTimeIndex 0  ' +  # Time index for the moving volume (0 means no time series)
                                    f'--numberOfHistogramBins 50  ' +  # Number of histogram bins for image matching
                                    f'--numberOfMatchPoints 10  ' +  # Number of points used in matching process
                                    f'--costMetric MMI  ' +  # Use Mutual Information (MMI) as the cost metric for registration
                                    f'--maskInferiorCutOffFromCenter 1000  ' +  # Masking option (cut off below 1000 units from center)
                                    f'--ROIAutoDilateSize 0  ' +  # Dilate ROI automatically (disabled)
                                    f'--ROIAutoClosingSize 9  ' +  # Apply morphological closing (dilation + erosion)
                                    f'--numberOfSamples 0  ' +  # Number of samples used in cost function (0 for all samples)
                                    f'--failureExitCode -1  ' +  # Exit code on failure
                                    f'--numberOfThreads -1  ' +  # Use all available threads
                                    f'--debugLevel 0  ' +  # Debug level (0 = no debug)
                                    f'--costFunctionConvergenceFactor 2e+13  ' +  # Convergence factor for cost function
                                    f'--projectedGradientTolerance 1e-05  ' +  # Tolerance for projected gradient
                                    f'--maximumNumberOfEvaluations 900  ' +  # Maximum number of evaluations for optimization
                                    f'--maximumNumberOfCorrections 25  ' +  # Maximum number of corrections for optimization
                                    f'--metricSamplingStrategy Random '  # Randomly sample points for metric calculation
                                    f'>> /dev/null'  # Suppress output to the terminal
                                       )
        
                # Check if the exit status is zero (success)
                if exit_status == 0:
                    print("BRAINSFit registration completed successfully!")
                else:
                    print(f"BRAINSFit failed with exit code {exit_status}. Please check for errors.")
                        
                # Apply the affine transformation to the target face mask and bring it into the space of the floating image
                os.system(f'./apply_affine "{target_face_mask}" "{floating}" "{out_affine}" "{results_folder_path}" -noshow')
                
                # Define path for the resampled face mask
                face_mask_resampled_path = os.path.join(results_folder_path, os.path.basename(target_face_mask).replace(".nii.gz", "_resampled.nii.gz"))
                
                # Resample the face mask to the floating image space using BRAINSresample
                os.system(      f' "{BRAINSresample_bin_path}" '  +  # BRAINSresample executable path
                                f'--inputVolume "{target_face_mask}" ' +  # Path to the face mask
                                f' --referenceVolume "{floating}" ' +  # Reference volume (floating image space)
                                f'--outputVolume "{face_mask_resampled_path}" ' +  # Path to the output resampled mask
                                f'--warpTransform "{out_affine}" ' +  # Use the previously computed affine transform
                                f'--inverseTransform ' +  # Apply the inverse transformation
                                f'--interpolationMode NearestNeighbor '  # Use nearest neighbor interpolation for resampling
                                # f'>> /dev/null'  # Suppress output to the terminal
                        )
        
                # Apply the resampled face mask to the floating image
                floating_masked_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))

                # Find the correct path to reorientated scans
                try:
                    brain_mask_path = floating_to_brain_mask[floating]
                except KeyError:
                    print(f"Warning: No corresponding whole scan found for {floating}. Skipping.")
                    continue

                # Reload the floating image as NIfTI objects
                floating_nii = nib.load(floating)
                floating_data = floating_nii.get_fdata()
                # Load face mask as NIfTI objects (defacing mask)
                face_mask_nii = nib.load(face_mask_resampled_path)
                face_mask_data = face_mask_nii.get_fdata()
                # Load face mask as NIfTI objects (skull-stripped mask from HD-BET)
                brain_mask_nii = nib.load(brain_mask_path)
                brain_mask_data = brain_mask_nii.get_fdata()

                # Combine face and brain masks
                mask_data = np.clip(face_mask_data + brain_mask_data, 0, 1)
        
                # Apply the mask to the floating image (element-wise multiplication)
                # floating_masked_data = floating_data * mask_data
                if floating_data.ndim == 4:  # Check if the floating image is 4D
                    print("4D case")
                    floating_masked_data = floating_data * mask_data[..., np.newaxis]  # Expand mask dimensions to match
                else:
                    floating_masked_data = floating_data * mask_data  # Standard 3D case
                            
                floating_masked_nii = nib.Nifti1Image(floating_masked_data, affine = floating_nii.affine)
            
                # Save the resulting masked floating image
                nib.save(floating_masked_nii, floating_masked_path)
    
        except Exception as e:
            print(f"Error processing {floating}: {e}")
            not_processed_scans.append(floating)

# Save list of scans not processed as CSV file (if needed)
if not_processed_scans:
    csv_path = os.path.join(destination_root, "not_defaced_scans.csv")
    with open(csv_path, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["None Defaced Scan Paths"])
        for path in not_processed_scans:
            writer.writerow([path])

    print(f"List of non-defaced scans saved to {csv_path}")
else:
    print("All scans were successfully processed.")


  0%|          | 0/2 [00:00<?, ?it/s]

0
./data/example_output_hdbet/output_brain_mask/IXI002-Guys-0828-T1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


Run registration with BRAINSFit...
BRAINSFit registration completed successfully!


 50%|#####     | 1/2 [01:46<01:46, 106.98s/it]

1
./data/example_output_hdbet/output_brain_mask/IXI002-Guys-0828-T2


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


Run registration with BRAINSFit...
BRAINSFit registration completed successfully!


100%|##########| 2/2 [05:50<00:00, 175.16s/it]


All scans were successfully processed.


### **Step 3.b: Affine Registration with Skull-Stripped Scan and Defacing**
In this alternative Step 3, each **dilated skull-stripped scan** is registered to the skull-stripped t1-w MNI152 template using **BRAINSFit**. If an existing transformation matrix (e.g., one generated in **3D Slicer**) is available, it will be used instead of performing the automatic registration process.  

The registration script takes the following inputs:  
- **Target image**: The skull-stripped t1-w MNI152 template (generated in the process).  
- **Target face mask**: A pre-defined face mask in MNI152 space.  
- **Floating images**: The dilated skull-stripped scans to be registered.  
- **Brain masks**: The dilated brain-only masks from **Step 2**.  
- **Existing transforms**: If available, previously computed transformation matrices.  
- **Reoriented Scans**: The reoriented scans from **Step 1** are used during the defacing process. A dictionary is created to map the skull-stripped scans (floating images during registration) to the corresponding reoriented scans (used as input for defacing).

For each scan, the following steps are performed:  
1. **Check for existing outputs** – If a defaced version already exists, the script skips reprocessing.  
2. **Perform registration** – If no existing transformation is available, BRAINSFit registers the scan to the template.  
3. **Apply the transformation** – The computed transformation is used to warp the **target face mask** into the scan’s space.  
4. **Resample the face mask** – The transformed face mask is adjusted to match the scan's resolution.  
5. **Combine masks** – The warped **face mask** and the **brain mask** from Step 2 are merged to create a final defacing mask.  
6. **Apply the mask** – The defacing mask is applied to the reoriented scan, removing facial features while preserving brain structures.  

A **log file** is created to track scans that could not be processed. If any scans fail, they will need to be manually registered using **alternative tools like 3D Slicer** and defaced.

In [ ]:
# Run once to obtain skull-stripped t1-w reference atlas
# Load t1-w reference atlas
t1_path = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/mni_icbm152_t1_tal_nlin_sym_55_ext.nii")
t1_nii = nib.load(t1_path)
t1_data = t1_nii.get_fdata()

# Load brain mask
brain_path = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/mni_icbm152_t1_tal_nlin_sym_55_ext_mask.nii")
brain_nii = nib.load(brain_path)
brain_data = brain_nii.get_fdata()

# Extract brain only region from t1-w scan
skull_stripped = t1_data * brain_data

# Save skull-stripped scan
skull_stripped_nii = nib.Nifti1Image(skull_stripped, affine=t1_nii.affine)
output_path = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/mni_icbm152_t1_tal_nlin_sym_55_ext_brain_only.nii")
nib.save(skull_stripped_nii, output_path)


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


In [35]:
# define the target image for the registration and the corresponding face mask
target = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/mni_icbm152_t1_tal_nlin_sym_55_ext_brain_only.nii")
target_face_mask = os.path.abspath("./data/icbm152_ext55_model_sym_2020_nifti/icbm152_ext55_model_sym_2020/t1_mask.nii.gz")

# define the list of images that will be used for registration (only dilated versions of hd-bet skull-stripped scans)
floating_imgs = natsorted([f for f in glob("./data/example_input_hdbet_processed/*/*.nii.gz") if "hd_bet_dilated_" in f])

# define a list of existing transforms (for example created with 3D Slicer) that are used instead of the automatic registration
# value should be None if no transform exists and automatic registration is to be used
existing_transform_paths = [os.path.join(os.path.dirname(f), "Transform_to_template.txt") for f in floating_imgs]
existing_transform_paths = [f if os.path.isfile(f) else "None" for f in existing_transform_paths ]

# list of output paths for each defaced image
destination_root = "./data/example_output_hdbet/output_skullstripped_registration"
results_folder_paths = [os.path.join(destination_root, *f.split(os.sep)[-2:-1]) for f in floating_imgs]

# check that all lists have the same length
assert len(floating_imgs) == len(results_folder_paths) == len(existing_transform_paths), f"The lists have different lengths, {len(floating_imgs)=}, {len(results_folder_paths)=}, {len(existing_transform_paths)=}"

print(f"Found {len(floating_imgs)} images that need to be defaced")

# print example paths
print("Input images: ", floating_imgs[0:3])
print("Existing transforms: ", existing_transform_paths[0:3])
print("Output paths: ", results_folder_paths[0:3])

# path to BRAINSFit executable
BRAINSFit_bin_path = os.path.join("./BRAINSTools/BRAINSFit")

# path to BRAINSFit executable
BRAINSresample_bin_path = "./BRAINSTools/BRAINSResample"

Found 2 images that need to be defaced
Input images:  ['./data/example_input_hdbet_processed/IXI002-Guys-0828-T1/hd_bet_dilated_IXI002-Guys-0828-T1.nii.gz', './data/example_input_hdbet_processed/IXI002-Guys-0828-T2/hd_bet_dilated_IXI002-Guys-0828-T2.nii.gz']
Existing transforms:  ['None', 'None']
Output paths:  ['./data/example_output_hdbet/output_skullstripped_registration/IXI002-Guys-0828-T1', './data/example_output_hdbet/output_skullstripped_registration/IXI002-Guys-0828-T2']


In [36]:
# Create a dictionary to pair dilated skull-stripped scans (input for registration) and reorientated whole scans (input for defacing)
# List the whole scan images with the 'reoriented_' prefix
whole_scan_imgs = natsorted(glob("./data/example_input_images/*/reoriented_*.nii.gz"))

# List the brain masks
brain_masks = natsorted(glob("./data/example_input_hdbet_processed/*/hd_bet_mask_*.nii.gz"))

# Create a dictionary to store pairs of floating images, whole scans, and corresponding brain masks
floating_to_whole_scan_and_mask = {}

# Iterate through the floating images and find the matching whole scan and brain mask for each
for floating_img in floating_imgs:
    # Remove the 'hd_bet_dilated_' prefix from the floating image filename
    base_name = os.path.basename(floating_img).replace("hd_bet_dilated_", "")
    
    # Search for the corresponding whole scan
    for whole_scan_img in whole_scan_imgs:
        if base_name in os.path.basename(whole_scan_img):
            # Find the corresponding brain mask
            for brain_mask in brain_masks:
                if base_name in os.path.basename(brain_mask):
                    # Store the floating image, whole scan, and brain mask in the dictionary
                    floating_to_whole_scan_and_mask[floating_img] = (whole_scan_img, brain_mask)
                    break  # Found the matching brain mask, no need to continue searching
            break  # Found the matching whole scan, no need to continue searching

# Print the resulting dictionary
print(floating_to_whole_scan_and_mask)


{'./data/example_input_hdbet_processed/IXI002-Guys-0828-T1/hd_bet_dilated_IXI002-Guys-0828-T1.nii.gz': ('./data/example_input_images/IXI002-Guys-0828-T1/reoriented_IXI002-Guys-0828-T1.nii.gz', './data/example_input_hdbet_processed/IXI002-Guys-0828-T1/hd_bet_mask_IXI002-Guys-0828-T1.nii.gz'), './data/example_input_hdbet_processed/IXI002-Guys-0828-T2/hd_bet_dilated_IXI002-Guys-0828-T2.nii.gz': ('./data/example_input_images/IXI002-Guys-0828-T2/reoriented_IXI002-Guys-0828-T2.nii.gz', './data/example_input_hdbet_processed/IXI002-Guys-0828-T2/hd_bet_mask_IXI002-Guys-0828-T2.nii.gz')}


In [ ]:
# Variable to store paths of files not defaced
not_processed_scans = []

# Iterate over each set of images, result folder paths, and existing transformation files using tqdm for progress tracking
for i, (floating, results_folder_path, ex_tfm) in enumerate(tqdm(list(zip(floating_imgs, results_folder_paths, existing_transform_paths)))):

        print(i)
        try:
            # Check if the masked version of the floating image already exists
            if os.path.isfile(os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))):
                # If the masked file already exists, print a message and skip processing this pair
                print("file exists. skipping ..... ")
                continue  # Skip to the next iteration (image pair)
        
            else:
                print(results_folder_path)
                # If the masked file does not exist, create the output directory for this image pair
                os.makedirs(results_folder_path, exist_ok=True)
        
                # Define paths for the affine transformation matrix and resampled floating image
                out_affine = os.path.join(results_folder_path, os.path.basename(floating.replace("nii.gz", "txt")))
                out_resampled_img_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_resampled.nii.gz"))
                
                # Load the target and floating images using nibabel (NIfTI format)
                target_nii = nib.load(target)
                floating_nii = nib.load(floating)
                
                # If there is an existing affine transformation, use it instead of running registration
                if os.path.isfile(ex_tfm):
                    print(f"Using existing initialTransform {ex_tfm} instead of running registration...")
                    out_affine = ex_tfm  # Use the existing affine transformation matrix
                else:
        
                    # If no existing affine transformation, perform image registration using BRAINSFit
                    print(f"Run registration with BRAINSFit...")
                    exit_status = os.system(f' "{BRAINSFit_bin_path}" ' +  # BRAINSFit executable path
                                    f'--fixedVolume "{target}" ' +  # Path to the fixed (target) volume
                                    f'--movingVolume "{floating}" ' +  # Path to the moving (floating) volume
                                    f'--outputVolume "{out_resampled_img_path}" ' +  # Path to the output resampled volume
                                    f'--outputTransform "{out_affine}" ' +  # Path to the output transformation matrix
                                    f'--samplingPercentage 0.1  ' +  # 10% of the data used for registration (lower values = faster, less accurate)
                                    f'--splineGridSize 14,10,12  ' +  # Grid size for spline transformation
                                    f'--initializeTransformMode useMomentsAlign  ' +  # Method to initialize the transformation
                                    f'--useRigid  ' +  # Apply rigid transformation (translation, rotation)
                                    f'--useAffine  ' +  # Apply affine transformation (scaling, shearing)
                                    f'--maskProcessingMode NOMASK  ' +  # Don't apply any masks during registration
                                    f'--medianFilterSize 0,0,0  ' +  # Disable median filtering (default 0)
                                    f'--removeIntensityOutliers 0  ' +  # Disable outlier removal
                                    f'--outputVolumePixelType float  ' +  # Set output pixel type to float
                                    f'--backgroundFillValue 0  ' +  # Background fill value set to 0 (empty space in images)
                                    f'--interpolationMode Linear  ' +  # Use linear interpolation for image resampling
                                    f'--numberOfIterations 1500  ' +  # Number of iterations for optimization
                                    f'--maximumStepLength 0.05  ' +  # Maximum step size for optimization
                                    f'--minimumStepLength 0.001  ' +  # Minimum step size for optimization
                                    f'--relaxationFactor 0.5  ' +  # Relaxation factor for optimization
                                    f'--translationScale 1000  ' +  # Scale for translation optimization
                                    f'--reproportionScale 1  ' +  # Scale for reproportioning optimization
                                    f'--skewScale 1  ' +  # Scale for skewness optimization
                                    f'--maxBSplineDisplacement 0  ' +  # Maximum displacement for B-spline transformation
                                    f'--fixedVolumeTimeIndex 0  ' +  # Time index for the fixed volume (0 means no time series)
                                    f'--movingVolumeTimeIndex 0  ' +  # Time index for the moving volume (0 means no time series)
                                    f'--numberOfHistogramBins 50  ' +  # Number of histogram bins for image matching
                                    f'--numberOfMatchPoints 10  ' +  # Number of points used in matching process
                                    f'--costMetric MMI  ' +  # Use Mutual Information (MMI) as the cost metric for registration
                                    f'--maskInferiorCutOffFromCenter 1000  ' +  # Masking option (cut off below 1000 units from center)
                                    f'--ROIAutoDilateSize 0  ' +  # Dilate ROI automatically (disabled)
                                    f'--ROIAutoClosingSize 9  ' +  # Apply morphological closing (dilation + erosion)
                                    f'--numberOfSamples 0  ' +  # Number of samples used in cost function (0 for all samples)
                                    f'--failureExitCode -1  ' +  # Exit code on failure
                                    f'--numberOfThreads -1  ' +  # Use all available threads
                                    f'--debugLevel 0  ' +  # Debug level (0 = no debug)
                                    f'--costFunctionConvergenceFactor 2e+13  ' +  # Convergence factor for cost function
                                    f'--projectedGradientTolerance 1e-05  ' +  # Tolerance for projected gradient
                                    f'--maximumNumberOfEvaluations 900  ' +  # Maximum number of evaluations for optimization
                                    f'--maximumNumberOfCorrections 25  ' +  # Maximum number of corrections for optimization
                                    f'--metricSamplingStrategy Random '  # Randomly sample points for metric calculation
                                    f'>> /dev/null'  # Suppress output to the terminal
                                       )
        
                # Check if the exit status is zero (success)
                if exit_status == 0:
                    print("BRAINSFit registration completed successfully!")
                else:
                    print(f"BRAINSFit failed with exit code {exit_status}. Please check for errors.")
                        
                # Apply the affine transformation to the target face mask and bring it into the space of the floating image
                os.system(f'./apply_affine "{target_face_mask}" "{floating}" "{out_affine}" "{results_folder_path}" -noshow')
                
                # Define path for the resampled face mask
                face_mask_resampled_path = os.path.join(results_folder_path, os.path.basename(target_face_mask).replace(".nii.gz", "_resampled.nii.gz"))
                
                # Resample the face mask to the floating image space using BRAINSresample
                os.system(      f' "{BRAINSresample_bin_path}" '  +  # BRAINSresample executable path
                                f'--inputVolume "{target_face_mask}" ' +  # Path to the face mask
                                f' --referenceVolume "{floating}" ' +  # Reference volume (floating image space)
                                f'--outputVolume "{face_mask_resampled_path}" ' +  # Path to the output resampled mask
                                f'--warpTransform "{out_affine}" ' +  # Use the previously computed affine transform
                                f'--inverseTransform ' +  # Apply the inverse transformation
                                f'--interpolationMode NearestNeighbor '  # Use nearest neighbor interpolation for resampling
                                # f'>> /dev/null'  # Suppress output to the terminal
                        )
        
                # Apply the resampled face mask to the floating image
                floating_masked_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))
        
                # Find the correct path to reorientated scans and binary mask
                try:
                    floating, brain_mask_path = floating_to_whole_scan_and_mask[floating]
                except KeyError:
                    print(f"Warning: No corresponding whole scan found for {floating}. Skipping.")
                    continue
                
                # Reload the floating image as NIfTI objects
                floating_nii = nib.load(floating)
                floating_data = floating_nii.get_fdata()
                # Load face mask as NIfTI objects (defacing mask)
                face_mask_nii = nib.load(face_mask_resampled_path)
                face_mask_data = face_mask_nii.get_fdata()
                # Load face mask as NIfTI objects (skull-stripped mask from HD-BET)
                brain_mask_nii = nib.load(brain_mask_path)
                brain_mask_data = brain_mask_nii.get_fdata()

                # Combine face and brain masks
                mask_data = np.clip(face_mask_data + brain_mask_data, 0, 1)
        
                # Apply the mask to the floating image (element-wise multiplication)
                # floating_masked_data = floating_data * mask_data
                if floating_data.ndim == 4:  # Check if the floating image is 4D
                    print("4D case")
                    floating_masked_data = floating_data * mask_data[..., np.newaxis]  # Expand mask dimensions to match
                else:
                    floating_masked_data = floating_data * mask_data  # Standard 3D case
                            
                floating_masked_nii = nib.Nifti1Image(floating_masked_data, affine = floating_nii.affine)
            
                # Save the resulting masked floating image
                nib.save(floating_masked_nii, floating_masked_path)
    
        except Exception as e:
            print(f"Error processing {floating}: {e}")
            not_processed_scans.append(floating)

# Save list of scans not processed as CSV file (if needed)
if not_processed_scans:
    csv_path = os.path.join(destination_root, "not_defaced_scans.csv")
    with open(csv_path, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["None Defaced Scan Paths"])
        for path in not_processed_scans:
            writer.writerow([path])

    print(f"List of non-defaced scans saved to {csv_path}")
else:
    print("All scans were successfully processed.")

  0%|          | 0/2 [00:00<?, ?it/s]

0
./data/example_output_hdbet/output_skullstripped_registration/IXI002-Guys-0828-T1
Run registration with BRAINSFit...
BRAINSFit registration completed successfully!


 50%|#####     | 1/2 [00:53<00:53, 53.10s/it]

1
./data/example_output_hdbet/output_skullstripped_registration/IXI002-Guys-0828-T2
Run registration with BRAINSFit...
BRAINSFit registration completed successfully!


100%|##########| 2/2 [01:33<00:00, 46.99s/it]


All scans were successfully processed.
